### Run this program at 9:18

In [1]:
from pya3 import *
import datetime as dt
import time

In [2]:
_id='308479'
apikey = '15NCRnTP02gQjlVS4C3EN0p6fhLVePQmD1xRhqKmMKf1EfSfQW5JIs74WKaZX74nXA2MPlU4574hOMd3DlhM4EWAOL6O3sq5M0X6EWh1XMHpQyB2kttXedHiEXqptCB5'

In [3]:
alice = Aliceblue(user_id=_id,api_key=apikey)

In [4]:
sessionid = alice.get_session_id() # Get Session ID

In [5]:
print(sessionid)

{'stat': 'Ok', 'sessionID': 'b80T8Tk9hfrL3PbbRGk2U1xZo70hzL98SxxcC9S0leCBTZ5nvCZiqTWpEvOpKVxcszJhQg2JYZECltHrjPsMMls69xXozzet9e5EOzdVvoZGrlQpLxEnm6iU62Wj0WDt2DAm11CGgH4Ii2nLRGKTvylYbeBiBXAWiPnecW1WFU8l0zathY7g5C2qTb3xaCoTv1QebRQfDBg0s26hUer8ELAe4GWb3B85g0L93LpFkQJ1Hrq7xQ4GElj9HnQWi9ZF'}


In [6]:
banknifty = alice.get_instrument_by_symbol('INDICES','NIFTY BANK')

In [7]:
###### Check trade Function
def check_trade():
    #alice=Aliceblue(_id,apikey)
    alice.get_session_id()['sessionID']
    signal = True
    days=1
    while(signal):
        try:
            exchange='INDICES'
            spot_symbol='NIFTY BANK'
            interval='D'
            indices=True
            from_date=dt.datetime.now()-dt.timedelta(days=days)
            to_date=dt.datetime.now()
            token=alice.get_instrument_by_symbol(exchange,spot_symbol)
            data=alice.get_historical(token,from_date,to_date,interval,indices)
            #print(data)
            y_high = data['high'][0]
            y_low = data['low'][0]
            today_open = data['open'][1]
            if(y_high > today_open and today_open > y_low):
                signal = False
                return True
            else:
                print("No Trade today")
                signal = False
                return False
        except:
            days = days + 1

In [8]:
Trade = check_trade()
print(Trade)

True


In [9]:
def get_curr_expiry(spot_ltp):
    spot_ltp = int(spot_ltp)
    print(f"Current expiry for {spot_ltp}")
    global datecale
    datecale = dt.date.today()
    while(1):
        expiry = alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date=str(datecale), is_fut=False,strike=spot_ltp, is_CE=True)
        #print(expiry)
        try:
            if list(expiry.values())[0] == 'Not_ok':
                print(f"{datecale} is not expiry")
                datecale = datecale + dt.timedelta(days=1)
            else:
                return datecale
                break
        except:
            return datecale

In [10]:
# get and Round off ltp on 09:20
# Entering into Trade
while(1):
    if (Trade == True):
        print(dt.datetime.now().time())
        if dt.datetime.now().time()>=dt.time(9,20,1):
            bnf_spot_ltp = float(alice.get_scrip_info(banknifty)['LTP'])
            round_bnf_spot_ltp = round((bnf_spot_ltp/100),0)*100
            bnf_expiry_date = get_curr_expiry(round_bnf_spot_ltp)
            ce_instrument = alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date=str(bnf_expiry_date),
                                                         is_fut=False,strike=round_bnf_spot_ltp, is_CE=True)
            pe_instrument = alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date=str(bnf_expiry_date),
                                                         is_fut=False,strike=round_bnf_spot_ltp, is_CE=False)
            cesell = float(alice.get_scrip_info(ce_instrument)['LTP'])#float(cedict['Prc'])
            pesell = float(alice.get_scrip_info(pe_instrument)['LTP'])#float(pedict['Prc'])
            if ((cesell + pesell > 272)):
                print(cesell + pesell)
                order1 = {  "instrument"        : ce_instrument,
                "order_type"        : OrderType.Market,
                "quantity"          : 25,
                "transaction_type"  : TransactionType.Sell,
                "product_type"      : ProductType.Intraday,
                "order_tag"         : "Order1"}
                order2 = {  "instrument"        : pe_instrument,
                "order_type"        : OrderType.Market,
                "quantity"          : 25,
                "transaction_type"  : TransactionType.Sell,
                "product_type"      : ProductType.Intraday,
                "order_tag"         : "Order2"}
                orders = [order1, order2]
                brorder = alice.place_basket_order(orders)
                print("Order placed")
                print(f"LTP is {bnf_spot_ltp} Sell PE and CE on {round_bnf_spot_ltp} Price at expiry {bnf_expiry_date}.")
                #print(ce_instrument,pe_instrument)
                break
            else:
                print("Not Trade today")
                break
        else:
            time.sleep(1)
    else:
        print("Not Trade today")
        break

09:18:14.122479
09:18:15.137064
09:18:16.145792
09:18:17.156695
09:18:18.164668
09:18:19.172748
09:18:20.183850
09:18:21.195146
09:18:22.207520
09:18:23.217347
09:18:24.228391
09:18:25.240286
09:18:26.249533
09:18:27.263165
09:18:28.275562
09:18:29.287453
09:18:30.302017
09:18:31.313692
09:18:32.323627
09:18:33.331785
09:18:34.340916
09:18:35.351495
09:18:36.365515
09:18:37.375789
09:18:38.386382
09:18:39.397266
09:18:40.404811
09:18:41.421073
09:18:42.435088
09:18:43.447391
09:18:44.460540
09:18:45.473584
09:18:46.473593
09:18:47.486203
09:18:48.495843
09:18:49.509199
09:18:50.520855
09:18:51.534959
09:18:52.547911
09:18:53.563112
09:18:54.573643
09:18:55.582920
09:18:56.594117
09:18:57.603415
09:18:58.613452
09:18:59.619956
09:19:00.634776
09:19:01.648711
09:19:02.659518
09:19:03.670461
09:19:04.679674
09:19:05.691117
09:19:06.705583
09:19:07.723387
09:19:08.730446
09:19:09.743160
09:19:10.753541
09:19:11.761704
09:19:12.763868
09:19:13.772230
09:19:14.782592
09:19:15.793231
09:19:16

In [11]:
## Calculating the Target and the StopLoss
cedict = alice.get_order_history(brorder[0]['NOrdNo'])
pedict = alice.get_order_history(brorder[1]['NOrdNo'])
cesell = float(alice.get_scrip_info(ce_instrument)['LTP'])#float(cedict['Prc'])
pesell = float(alice.get_scrip_info(pe_instrument)['LTP'])#float(pedict['Prc'])
target = (cesell + pesell) * 0.9
stoploss = (cesell + pesell) * 1.10
print("Target: ", target)
print("Stoploss: ",stoploss)
print("Value Sum: ",cesell + pesell)

Target:  455.67
Stoploss:  556.9300000000001
Value Sum:  506.3


In [14]:
## Calculating the trade 
while(1):
    time.sleep(5)
    celtp = float(alice.get_scrip_info(ce_instrument)['LTP'])
    peltp = float(alice.get_scrip_info(pe_instrument)['LTP'])
    if ((celtp + peltp) <= target):
        print("Target Achive Square Off the Trade at: ", celtp + peltp)
        order3 = {  "instrument"        : ce_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order3"}
        order4 = {  "instrument"        : pe_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order4"}
        orders = [order3, order4]
        brorder = alice.place_basket_order(orders)
        break
    elif (celtp + peltp) >= stoploss:
        print("StopLoss Hit Square Off the Trade at: ", celtp + peltp)
        order3 = {  "instrument"        : ce_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order3"}
        order4 = {  "instrument"        : pe_instrument,
        "order_type"        : OrderType.Market,
        "quantity"          : 25,
        "transaction_type"  : TransactionType.Buy,
        "product_type"      : ProductType.Intraday,
        "order_tag"         : "Order4"}
        orders = [order3, order4]
        brorder = alice.place_basket_order(orders)
        break
    else:
        try :
            print(f"Current Sum: {celtp + peltp} and Running Profit/Loss: {(cesell + pesell)-(celtp + peltp)}")
        except:
            print("Not get LTP")
            time.sleep(10)

Current Sum: 522.35 and Running Profit/Loss: -16.05000000000001
Current Sum: 521.5 and Running Profit/Loss: -15.199999999999989
Current Sum: 521.75 and Running Profit/Loss: -15.449999999999989
Current Sum: 520.05 and Running Profit/Loss: -13.749999999999943
Current Sum: 519.85 and Running Profit/Loss: -13.550000000000011
Current Sum: 519.9 and Running Profit/Loss: -13.599999999999966
Current Sum: 519.35 and Running Profit/Loss: -13.050000000000011
Current Sum: 517.75 and Running Profit/Loss: -11.449999999999989
Current Sum: 517.8 and Running Profit/Loss: -11.499999999999943
Current Sum: 517.8 and Running Profit/Loss: -11.499999999999943
Current Sum: 518.4000000000001 and Running Profit/Loss: -12.10000000000008
Current Sum: 518.3 and Running Profit/Loss: -11.999999999999943
Current Sum: 516.8 and Running Profit/Loss: -10.499999999999943
Current Sum: 516.45 and Running Profit/Loss: -10.150000000000034
Current Sum: 516.5 and Running Profit/Loss: -10.199999999999989
Current Sum: 515.599999

Current Sum: 521.3 and Running Profit/Loss: -14.999999999999943
Current Sum: 520.7 and Running Profit/Loss: -14.400000000000034
Current Sum: 521.45 and Running Profit/Loss: -15.150000000000034
Current Sum: 521.35 and Running Profit/Loss: -15.050000000000011
Current Sum: 520.6 and Running Profit/Loss: -14.300000000000011
Current Sum: 519.0 and Running Profit/Loss: -12.699999999999989
Current Sum: 520.45 and Running Profit/Loss: -14.150000000000034
Current Sum: 520.3 and Running Profit/Loss: -13.999999999999943
Current Sum: 519.5 and Running Profit/Loss: -13.199999999999989
Current Sum: 518.85 and Running Profit/Loss: -12.550000000000011
Current Sum: 516.4 and Running Profit/Loss: -10.099999999999966
Current Sum: 516.7 and Running Profit/Loss: -10.400000000000034
Current Sum: 515.7 and Running Profit/Loss: -9.400000000000034
Current Sum: 514.3 and Running Profit/Loss: -7.999999999999943
Current Sum: 516.25 and Running Profit/Loss: -9.949999999999989
Current Sum: 517.7 and Running Profit/

Current Sum: 513.2 and Running Profit/Loss: -6.900000000000034
Current Sum: 516.1 and Running Profit/Loss: -9.800000000000011
Current Sum: 513.9 and Running Profit/Loss: -7.599999999999966
Current Sum: 513.1 and Running Profit/Loss: -6.800000000000011
Current Sum: 513.05 and Running Profit/Loss: -6.749999999999943
Current Sum: 513.3499999999999 and Running Profit/Loss: -7.049999999999898
Current Sum: 513.9000000000001 and Running Profit/Loss: -7.60000000000008
Current Sum: 514.2 and Running Profit/Loss: -7.900000000000034
Current Sum: 514.1500000000001 and Running Profit/Loss: -7.85000000000008
Current Sum: 514.1 and Running Profit/Loss: -7.800000000000011
Current Sum: 515.7 and Running Profit/Loss: -9.400000000000034
Current Sum: 515.75 and Running Profit/Loss: -9.449999999999989
Current Sum: 516.15 and Running Profit/Loss: -9.849999999999966
Current Sum: 514.55 and Running Profit/Loss: -8.249999999999943
Current Sum: 514.7 and Running Profit/Loss: -8.400000000000034
Current Sum: 515.

Current Sum: 517.2 and Running Profit/Loss: -10.900000000000034
Current Sum: 517.95 and Running Profit/Loss: -11.650000000000034
Current Sum: 517.4 and Running Profit/Loss: -11.099999999999966
Current Sum: 518.2 and Running Profit/Loss: -11.900000000000034
Current Sum: 519.65 and Running Profit/Loss: -13.349999999999966
Current Sum: 520.05 and Running Profit/Loss: -13.749999999999943
Current Sum: 524.5 and Running Profit/Loss: -18.19999999999999
Current Sum: 524.65 and Running Profit/Loss: -18.349999999999966
Current Sum: 524.65 and Running Profit/Loss: -18.349999999999966
Current Sum: 522.85 and Running Profit/Loss: -16.55000000000001
Current Sum: 523.8 and Running Profit/Loss: -17.499999999999943
Current Sum: 523.7 and Running Profit/Loss: -17.400000000000034
Current Sum: 524.15 and Running Profit/Loss: -17.849999999999966
Current Sum: 523.25 and Running Profit/Loss: -16.94999999999999
Current Sum: 524.85 and Running Profit/Loss: -18.55000000000001
Current Sum: 529.25 and Running Pro

Current Sum: 524.2 and Running Profit/Loss: -17.900000000000034
Current Sum: 524.1 and Running Profit/Loss: -17.80000000000001
Current Sum: 524.85 and Running Profit/Loss: -18.55000000000001
Current Sum: 525.4 and Running Profit/Loss: -19.099999999999966
Current Sum: 526.8 and Running Profit/Loss: -20.499999999999943
Current Sum: 527.2 and Running Profit/Loss: -20.900000000000034
Current Sum: 528.4000000000001 and Running Profit/Loss: -22.10000000000008
Current Sum: 527.65 and Running Profit/Loss: -21.349999999999966
Current Sum: 529.5 and Running Profit/Loss: -23.19999999999999
Current Sum: 528.45 and Running Profit/Loss: -22.150000000000034
Current Sum: 527.0 and Running Profit/Loss: -20.69999999999999
Current Sum: 525.6500000000001 and Running Profit/Loss: -19.35000000000008
Current Sum: 527.0 and Running Profit/Loss: -20.69999999999999
Current Sum: 528.8 and Running Profit/Loss: -22.499999999999943
Current Sum: 533.0 and Running Profit/Loss: -26.69999999999999
Current Sum: 533.0 an